# Pandas3 : 그룹별 집계

In [21]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")


---

### 1.그룹별 집계의 개념
판다스의 중요한 기능 중 하나는 **그룹별 집계(groupby)** 이다. 그룹별 집계는 데이터로부터 동일한 객체를 가진 데이터만 따로 뽑아 기술통계 데이터를 추출하는 기능이다. groupby 명령어는 분할->적용->결합 과정을 거친다.

● **분할(split)** : 데이터에서 같은 종류의 데이터끼리 나누는 기능. 데이터를 나누는 기준은 key값이 되며, 다양한 종류의 열을 사용할 수 있음.  
● **적용(apply)** : 데이터 분할로 나누어진 데이터 블록마다 다양한 연산 적용.  
● **결합(combine)** : 데이터 분하로 나누어진 블록마다 연산 적용으로 인해 함수가 적용된 각 블록을 합치는 단계.

---

### 2.그룹별 집계 사용

#### 2-1.그룹별 집계의 기본형  
그룹별 집계를 사용하기 위해 다음 코드와 같은 데이터를 준비한다. 데이터는 레이싱 팀이 매년 경기에 참가하여 얻은 순위와 획득점수를 정리한 데이터 테이블이다.  

In [5]:
data={'Team':['Riders','Riders','Devils','Devils','Kings','Kings','Kings','Kings','Riders','Royals','Royals','Riders'],
      'Rank':[1,2,2,3,3,4,1,1,2,4,1,2],
      'Year':[2014,2015,2014,2014,2014,2015,2016,2017,2016,2014,2015,2017],
      'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}


df=pd.DataFrame(data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2014,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [ ]:
## Team이라는 키값으로 묶고 Points 열을 추출하여 sum함수를 적용
print(df.groupby('Team')['Points'].sum())

Team
Devils    1536
Kings     3097
Riders    3049
Royals    1505
Name: Points, dtype: int64


한 가지 기억할 점은 해당 결과가 데이터프레임 객체가 아닌 **시리즈** 객체로 반환되기 때문에 데이터 타입을 다루는데 유의해야 한다.

#### 2-2.멀티 인덱스 그룹별 집계
앞에서는 한 개의 열을 기준을 그룹별 집계를 실행했는데, 실제로는 한 개 이상의 열을 기준으로 그룹별 집계를 실행하는 경우가 있다. 한개 이상의 열을 넣어야 할때에는 리스트를 사용하여 여러 개의 열 이름을 넣을 수 있다.  
이렇게 출력된 결과물 역시 시리즈 객체이다. 다시 데이터프레임으로 변형하기 위해선 reset_index 함수를 적용해야 한다.

In [9]:
print(df.groupby(['Team','Year'])['Points'].sum())

Team    Year
Devils  2014    1536
Kings   2014     741
        2015     812
        2016     756
        2017     788
Riders  2014     876
        2015     789
        2016     694
        2017     690
Royals  2014     701
        2015     804
Name: Points, dtype: int64


#### 2-3.멀티 인덱스
그룹별 집계를 사용하다 보면 한 개 이상의 인덱스를 갖는 시리즈 객체가 출력된다. 이것은 한 개 이상의 열을 사용하여 그룹별 집계를 수행하게 되면 해당 열들이 모두 인덱스로 변환되기 때문인데, 이를 **멀티 인덱스**라고 부른다.  
이러한 멀티 인덱스는 다음 코드와 같이 **인덱스 요소(index property)** 를 이용해서 확인할 수 있다. 멀티 인덱스를 사용하여 인덱스를 추출할 수 있다.

In [12]:
multi_groupby=df.groupby(['Team','Year'])['Points'].sum()
multi_groupby.index

MultiIndex([('Devils', 2014),
            ( 'Kings', 2014),
            ( 'Kings', 2015),
            ( 'Kings', 2016),
            ( 'Kings', 2017),
            ('Riders', 2014),
            ('Riders', 2015),
            ('Riders', 2016),
            ('Riders', 2017),
            ('Royals', 2014),
            ('Royals', 2015)],
           names=['Team', 'Year'])

In [13]:
multi_groupby['Devils':'Kings']

Team    Year
Devils  2014    1536
Kings   2014     741
        2015     812
        2016     756
        2017     788
Name: Points, dtype: int64

**unstack** 함수를 사용하여 기존 인덱스를 기준으로 묶인 데이터에서 두 번째 인덱스를 열로 변화시켜 엑셀의 피벗테이블과 유사한 형태로 데이터를 출력한다.

In [14]:
multi_groupby.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,1536.0,NaN,NaN,NaN
Kings,741.0,812.0,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN


필요에 따라 다음 코드와 같이 **swaplevel** 함수를 이용하면 인덱스 간의 레벨을 변경할 수 있다. 추가적으로 **sort_index** 함수를 사용하면 변경된 인덱스 중 첫 번째 인덱스를 기준으로 데이터를 재정렬해준다.

In [15]:
multi_groupby.swaplevel().sort_index()

Year  Team  
2014  Devils    1536
      Kings      741
      Riders     876
      Royals     701
2015  Kings      812
      Riders     789
      Royals     804
2016  Kings      756
      Riders     694
2017  Kings      788
      Riders     690
Name: Points, dtype: int64

다음과 같이 각 레벨별로 별도의 연산 함수를 적용할 수 있다.

In [22]:
multi_groupby.sum(level=0)

Team
Devils    1536
Kings     3097
Riders    3049
Royals    1505
Name: Points, dtype: int64

In [23]:
multi_groupby.sum(level=1)

Year
2014    3854
2015    2405
2016    1450
2017    1478
Name: Points, dtype: int64

---

### 3.그룹화 상태
**분할->적용->결합**으로 이어지는 groupby 함수 사용에서 분할만 적용한 상태로 데이터를 다룰 수 있는데, 이를 **그룹화(grouped)** 상태라고 한다. grouped 함수의 사용은 **get_group** 함수를 사용하면 해당 키 값을 기준으로 분할된 데이터프레임 객체를 확인할 수 있다. 그룹화된 상태의 데이터프레임에 다음과 같은 기능을 적용할 수 있다.

● **집계(aggregation)** : 요약 통계 추출  

● **변환(transformation)** : 해당 정보를 변환  

● **필터(filter)** : 특정 정보를 제거하여 보여주는 필터링 

In [20]:
grouped = df.groupby('Team')
grouped.get_group('Riders')

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
8,Riders,2,2016,694
11,Riders,2,2017,690


#### 3-1.집계
집계를 적용하기 위해서는 agg 함수를 사용한다. agg는 다양한 함수를 그대로 적용시킨다는 측면에서 단순히 groupby를 사용했을 때의 결과와 크게 차이가 없다.

In [24]:
grouped.agg(min)

,Rank,Year,Points
Team,,,
Devils,2,2014,673
Kings,1,2014,741
Riders,1,2014,690
Royals,1,2014,701


#### 3-2.변환
변환은 집계와 달리 키 값별로 요약된 정보가 아닌 개별 데이터의 변환을 지원한다. 그러한 적용하는 시점에서는 그룹화된 상태의 값으로 적용된다. 이때 유용하게 사용할 수 있는 기법 중 하나는 그룹화된 상태에서 통계 정보가 필요할 때 다음 코드와 같이 **transform** 함수를 사용하는 것이다.

In [26]:
grouped.transform(max)

,Rank,Year,Points
0,2,2017,876
1,2,2017,876
2,3,2014,863
3,3,2014,863
4,4,2017,812
5,4,2017,812
6,4,2017,812
7,4,2017,812
8,2,2017,876
9,4,2015,804


In [27]:
score = lambda x:(x-x.mean()) / x.std()
grouped.transform(score)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,NaN,0.707107
3,0.707107,NaN,-0.707107
4,0.500000,-1.161895,-1.042333
5,1.166667,-0.387298,1.183401
6,-0.833333,0.387298,-0.572108
7,-0.833333,1.161895,0.431040
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


#### 3-3.필터
**필터(filter)** 는 특정 조건으로 데이터를 검색할 때 사용하는 함수이다. 주로 filter 함수를 사용하는데, 여기서 x가 의미하는 것은 분할(split)된 상태에서 각각의 그룹화된 데이터프레임을 의미한다.

In [28]:
df.groupby('Team').filter(lambda x: len(x) >=3 )

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


In [29]:
df.groupby('Team').filter(lambda x: x['Points'].max()>800)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2014,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701
